In [1]:
from z3 import *

In [2]:
from logic import Vocab

In [3]:
vocab = Vocab()

pnts = [vocab.fresh_pt() for _ in range(5)]

constr_range = [And(0 <= x, x < 5, 0 <= y, y < 3) for x, y in pnts]

In [4]:
from logic import eq, neq, lt, ptin, RelationOps

constr_distinct = [lt(p1, p2) for i in range(len(pnts)) for p1 in [pnts[i]] for p2 in pnts[i+1:]]
constr_distinct

[Or(x$1 < x$3, And(x$1 == x$3, y$2 < y$4)),
 Or(x$1 < x$5, And(x$1 == x$5, y$2 < y$6)),
 Or(x$1 < x$7, And(x$1 == x$7, y$2 < y$8)),
 Or(x$1 < x$9, And(x$1 == x$9, y$2 < y$10)),
 Or(x$3 < x$5, And(x$3 == x$5, y$4 < y$6)),
 Or(x$3 < x$7, And(x$3 == x$7, y$4 < y$8)),
 Or(x$3 < x$9, And(x$3 == x$9, y$4 < y$10)),
 Or(x$5 < x$7, And(x$5 == x$7, y$6 < y$8)),
 Or(x$5 < x$9, And(x$5 == x$9, y$6 < y$10)),
 Or(x$7 < x$9, And(x$7 == x$9, y$8 < y$10))]

In [5]:
constr_justify = [Or(p[0] == 0 for p in pnts), Or(p[1] == 0 for p in pnts)]
constr_justify

[Or(x$1 == 0, x$3 == 0, x$5 == 0, x$7 == 0, x$9 == 0),
 Or(y$2 == 0, y$4 == 0, y$6 == 0, y$8 == 0, y$10 == 0)]

In [6]:
def adj(p1, p2):
    (x1, y1), (x2, y2) = p1, p2
    return Or(And(x1 == x2, Or(y1 == y2 + 1, y2 == y1 + 1)),
              And(y1 == y2, Or(x1 == x2 + 1, x2 == x1 + 1)))


comp = RelationOps(vocab).comp

def path(p1, p2, lp):
    r = lambda p1, p2: Or(eq(p1, p2), And(adj(p1, p2), ptin(p2, lp)))
    r5 = comp(r, comp(r, comp(r, r)))
    return r5(p1, p2)

constr_contig = [path(p1, p2, pnts) for p1 in pnts for p2 in pnts if p1 is not p2]
#constr_contig

In [7]:
s = Solver()
s.add(*constr_range, *constr_distinct, *constr_justify, *constr_contig)

sols = []

for i in range(100):
    if s.check() != sat: break
    m = s.model()
    sol = [(m[x].as_long(), m[y].as_long()) for x,y in pnts]
    #print(sol)
    sols.append(sol)
    s.add(Not(And(v() == m[v] for v in m.decls())))



In [8]:
from penta import Penta
from presentation_forms import table_repr, vertically

def uniq_by(coll, eq=lambda x,y: x == y):
    uniq = []
    for p in coll:
        if not any(eq(q, p) for q in uniq):
            uniq.append(p)
    return uniq

pentas = uniq_by([Penta(sol) for sol in sols], lambda p, q: p.like(q))

table_repr([pentas])

,,,,,,,,,,,


In [9]:
import pickle
with open('../../data/pentas.pickle', 'wb') as outf:
    pickle.dump(pentas, outf)